In [1]:
import os
import subprocess
import importlib
import pandas as pd
import tokenize

In [2]:
import create_data_set_utils
importlib.reload(create_data_set_utils)

<module 'create_data_set_utils' from '/home/erez/research/code_completion/create_data_set_utils.py'>

In [3]:
dir_path = r"/home/erez/github_repos/siva/latest/"

In [4]:
# get_data_set.unpack_siva_files_and_checkout_git_repos(dir_path)

In [5]:
all_python_files_list = create_data_set_utils.get_python_files_paths(dir_path)

In [6]:
len(all_python_files_list)

10035

In [7]:
python_files_df = create_data_set_utils.create_python_files_df(all_python_files_list)

In [8]:
python_files_df.head()

,file_path,file_base_name,file_size,number_of_lines
0,/home/erez/github_repos/siva/latest/e2/temp_gi...,get_svhn.py,2033,11
1,/home/erez/github_repos/siva/latest/e2/temp_gi...,load_svhn.py,5840,39
2,/home/erez/github_repos/siva/latest/e2/temp_gi...,test.py,49,26
3,/home/erez/github_repos/siva/latest/e2/temp_gi...,utils.py,108,11
4,/home/erez/github_repos/siva/latest/e2/temp_gi...,loader.py,893,10


In [9]:
python_files_df['number_of_lines'].sum()

309690

In [10]:
tokens_df = create_data_set_utils.create_tokens_df(python_files_df)
tokens_df.head()

/home/erez/github_repos/siva/latest/b1/temp_git_repo_b1cedd8ed2f705506c1e0d9e13dbfbb340741329.siva/2.Forecast_name/main.py
/home/erez/github_repos/siva/latest/37/temp_git_repo_377c20db71d269f3d2d958d51d8435691c679afe.siva/model/APV_MCTS_tree.py
/home/erez/github_repos/siva/latest/37/temp_git_repo_377c20db71d269f3d2d958d51d8435691c679afe.siva/support/go-NN-master/engine/OnlineExampleMaker.py
/home/erez/github_repos/siva/latest/25/temp_git_repo_2560481619759357036b065cf0f7d599b7ac7638.siva/extract_features.py


,end,file_path,line,line_index,start,string,type_name
0,"(0, 0)",/home/erez/github_repos/siva/latest/e2/temp_gi...,,0,"(0, 0)",utf-8,BACKQUOTE
1,"(1, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys\n,1,"(1, 0)",import,NAME
2,"(1, 10)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys\n,1,"(1, 7)",sys,NAME
3,"(1, 11)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys\n,1,"(1, 10)",\n,NEWLINE
4,"(2, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import os\n,2,"(2, 0)",import,NAME


In [11]:
tokens_df.shape

(12500373, 7)

In [12]:
tokens_df.memory_usage(deep=True).sum()

3940934381

In [13]:
len(tokens_df['string'].unique())

321686

In [14]:
tokens_df['type_name'].unique().tolist()

['BACKQUOTE',
 'NAME',
 'NEWLINE',
 'OP',
 'NL',
 'INDENT',
 'STRING',
 'NUMBER',
 'DEDENT',
 'COMMENT',
 'ENDMARKER']

In [15]:
tokens_df.groupby('type_name').size()

type_name
BACKQUOTE      10031
COMMENT       158895
DEDENT        223791
ENDMARKER      10031
INDENT        223791
NAME         4175202
NEWLINE       949939
NL            655768
NUMBER        501763
OP           5215984
STRING        375178
dtype: int64

In [16]:
tokens_df.groupby('type_name').apply(lambda x: len(x['string'].unique()))

type_name
BACKQUOTE         4
COMMENT       71481
DEDENT            1
ENDMARKER         1
INDENT           54
NAME         125697
NEWLINE           3
NL                3
NUMBER         7220
OP               44
STRING       117184
dtype: int64

In [17]:
tokens_df[tokens_df['type_name'] == 'NEWLINE']['string'].value_counts()

\n      932386
\r\n     16632
           921
Name: string, dtype: int64

In [18]:
tokens_df[tokens_df['type_name'] == 'OP']['string'].value_counts()

,      958817
.      908706
(      841036
)      841036
=      635834
:      296297
[      250596
]      250596
+       37337
-       35671
*       29186
==      25251
{       18199
}       18199
%       13696
/       10104
+=       7848
@        7045
**       6193
>        5469
!=       4463
<        3773
>=       1889
//       1588
;        1584
<=       1279
*=        637
-=        563
&         545
|         519
/=        479
>>        378
<<        366
...       343
~         281
|=         59
->         30
^          28
&=         23
//=        22
%=          7
>>=         6
^=          5
**=         1
Name: string, dtype: int64

In [19]:
tokens_df[tokens_df['type_name'] == 'BACKQUOTE']['string'].value_counts()

utf-8    10003
utf8        25
gbk          2
cp949        1
Name: string, dtype: int64

In [20]:
tokens_df[tokens_df['type_name'] == 'INDENT']['string'].value_counts()

                                                    61357
                                                    54183
                                                    38589
                                                    23444
                                                    15793
                                                    14243
                                                     5788
                                                     3807
                                                     1982
\t\t                                                 1129
\t                                                    744
\t\t\t                                                726
                                                      555
                                                      389
\t\t\t\t                                              380
\t\t\t\t\t                                            148
                                                      136
              

In [21]:
tokens_df[tokens_df['type_name'] == 'DEDENT']['string'].value_counts()

    223791
Name: string, dtype: int64

In [22]:
tokens_df[tokens_df['type_name'] == 'NL']['string'].value_counts()

\n      647654
\r\n      8078
            36
Name: string, dtype: int64

In [23]:
python_files_df.to_csv('python_files.csv')

In [24]:
# tokens_df.to_csv('tokens.csv')

In [25]:
tokens_df[tokens_df['type_name'] == 'NAME']['string'].value_counts()[:50]

self           288038
tf              97244
def             81524
if              62402
import          59563
np              50895
return          48396
None            45089
in              42726
for             36439
x               35525
from            31573
name            30438
True            24156
False           23790
i               23634
with            22315
as              21586
result          19901
dtype           18928
df              18806
else            18390
data            18040
len             17858
nn              16501
index           16088
torch           15278
not             15276
assertEqual     15262
shape           14913
print           14647
os              14299
path            14119
expected        13936
args            13823
sess            13686
batch_size      13150
model           11419
class           10810
append          10801
tm              10593
is              10529
s               10327
run              9873
y                9796
size      